In [4]:
# speed test
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import torch
import pandas as pd
import os
from models.models import ConvLSTM, InceptionV3Model, ViTModel
from utils.preprocess import prepare_test_loader
import time

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the test dataset
dataset_path = './utils/data/test_labels.csv'
test_df = pd.read_csv(dataset_path)

min_max_file = './utils/min_max_values.csv'

min_max_df = pd.read_csv(min_max_file)

dataset_path = '../data/nutrition5k_reconstructed/'

image_path = os.path.join(dataset_path, 'images')

def speed_test(model, model_path):
    model = model.to(device)
    model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
    
    IMG_DIM = 299 if isinstance(model, InceptionV3Model) else 224
    
    test_loader = prepare_test_loader(test_df, image_path, ["calories", "mass", "fat", "carb", "protein"] , img_dim=IMG_DIM, batch_size=16)

    model.eval()
    start_time = time.time()
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
    end_time = time.time()
    print(f"Time taken: {end_time - start_time} seconds")
    print(f"Time taken per image: {(end_time - start_time) / len(test_df)} seconds")
    
tasks = ["calories", "mass", "fat", "carb", "protein"]
speed_test(ConvLSTM(tasks), './models/checkpoints/convlstm_log.pth')
speed_test(InceptionV3Model(tasks), './models/checkpoints/inceptionv3_log_regular.pth')
speed_test(ViTModel(tasks), './models/checkpoints/vit_log_regular.pth')

Time taken: 11.66094708442688 seconds
Time taken per image: 0.038358378567193686 seconds
Time taken: 15.795039653778076 seconds
Time taken per image: 0.051957367282164726 seconds
Time taken: 17.820536851882935 seconds
Time taken per image: 0.05862018701277281 seconds
